In [3]:
import numpy as np
import time
from amc_parser import *
import os
import sys
from pathlib import Path

In [18]:
#abs_path = os.getcwd()
asf_path = 'data\\124.asf'
asm_path = 'data\\124_01.amc'
joints = parse_asf(asf_path)

In [25]:
joints['head']

In [15]:
p=Path(abs_path)
q = p / 'data/124.asf'
#parse_asf(q)

{'head': <amc_parser.Joint at 0x1e612d0b518>,
 'lclavicle': <amc_parser.Joint at 0x1e612d0b550>,
 'lfemur': <amc_parser.Joint at 0x1e612d04f28>,
 'lfingers': <amc_parser.Joint at 0x1e612d0b6d8>,
 'lfoot': <amc_parser.Joint at 0x1e612d04fd0>,
 'lhand': <amc_parser.Joint at 0x1e612d0b588>,
 'lhipjoint': <amc_parser.Joint at 0x1e612d04e48>,
 'lhumerus': <amc_parser.Joint at 0x1e612d0b128>,
 'lowerback': <amc_parser.Joint at 0x1e612d0b2e8>,
 'lowerneck': <amc_parser.Joint at 0x1e612d0b400>,
 'lradius': <amc_parser.Joint at 0x1e612d0b5f8>,
 'lthumb': <amc_parser.Joint at 0x1e612d0b630>,
 'ltibia': <amc_parser.Joint at 0x1e612d04ef0>,
 'ltoes': <amc_parser.Joint at 0x1e612d04f98>,
 'lwrist': <amc_parser.Joint at 0x1e612d0b6a0>,
 'rclavicle': <amc_parser.Joint at 0x1e612d0b748>,
 'rfemur': <amc_parser.Joint at 0x1e612d0b198>,
 'rfingers': <amc_parser.Joint at 0x1e612d0b7f0>,
 'rfoot': <amc_parser.Joint at 0x1e612d04f60>,
 'rhand': <amc_parser.Joint at 0x1e612d0b898>,
 'rhipjoint': <amc_parser